In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 day 1 hour 39 mins
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,1 month and 13 days
H2O cluster name:,H2O_from_python_ec0198_unkim7
H2O cluster total nodes:,1
H2O cluster free memory:,2.685 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [2]:
file1="http://coursera.h2o.ai/cacao.882.csv"

In [3]:
rating= h2o.import_file(file1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
rating.head()

Maker,Origin,REF,Review Date,Cocoa Percent,Maker Location,Rating,Bean Type,Bean Origin
A. Morin,Agua Grande,1876,2016,0.63,France,3.75,,Sao Tome
A. Morin,Kpime,1676,2015,0.7,France,2.75,,Togo
A. Morin,Atsane,1676,2015,0.7,France,3,,Togo
A. Morin,Akata,1680,2015,0.7,France,3.5,,Togo
A. Morin,Quilla,1704,2015,0.7,France,3.5,,Peru
A. Morin,Carenero,1315,2014,0.7,France,2.75,Criollo,Venezuela
A. Morin,Cuba,1315,2014,0.7,France,3.5,,Cuba
A. Morin,Sur del Lago,1315,2014,0.7,France,3.5,Criollo,Venezuela
A. Morin,Puerto Cabello,1319,2014,0.7,France,3.75,Criollo,Venezuela
A. Morin,Pablino,1319,2014,0.7,France,4,,Peru


In [5]:
rating.summary()

,Maker,Origin,REF,Review Date,Cocoa Percent,Maker Location,Rating,Bean Type,Bean Origin
type,enum,enum,int,int,real,enum,real,enum,enum
mins,,,5.0,2006.0,0.42,,1.0,,
mean,,,1035.90473538,2012.32534819,0.716983286908,,3.18593314763,,
maxs,,,1952.0,2017.0,1.0,,5.0,,
sigma,,,552.88636501,2.92721027767,0.0632311833843,,0.478062393586,,
zeros,,,0,0,0,,0,,
missing,0,0,0,0,0,0,0,1,1
0,A. Morin,Agua Grande,1876.0,2016.0,0.63,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676.0,2015.0,0.7,France,2.75,,Togo
2,A. Morin,Atsane,1676.0,2015.0,0.7,France,3.0,,Togo


In [6]:
rating.nrows

1795

In [7]:
rating_maker=rating.group_by('Maker')

In [8]:
rating_maker.count()
rating_maker.get_frame()

Maker,nrow
A. Morin,23
AMMA,5
Acalli,2
Adi,4
Aequare (Gianduja),2
Ah Cacao,1
Akesson's (Pralus),3
Alain Ducasse,5
Alexandre,4
Altus aka Cao Artisan,10


In [9]:
# spliting train,test and valid data sets
train, test,valid = rating.split_frame(
    ratios=[0.81,0.09],
    destination_frames=['rating_train','rating_valid','rating_test'],seed = 122)

In [10]:
print ("%d/%d/%d" % (train.nrows,valid.nrows,test.nrows))

1452/176/167


In [11]:
#declaring predictor variable and excluding ID's
y = "Rating"
ignoreFields =[y]

x= [i for i in train.names if i not in ignoreFields]

In [12]:
x

[u'Maker',
 u'Origin',
 u'REF',
 u'Review Date',
 u'Cocoa Percent',
 u'Maker Location',
 u'Bean Type',
 u'Bean Origin']

In [13]:

from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [14]:
glm_gauss = H2OGeneralizedLinearEstimator(
                    model_id='glm_v1',            #allows us to easily locate this model in Flow
                    family='gaussian',seed=122, alpha=0.5,nfolds=5)


In [15]:
glm_gauss.train(x,y, train, validation_frame = valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [16]:
glm_gauss

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  glm_v1


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.207466316266
RMSE: 0.455484704756
MAE: 0.366220890926
RMSLE: 0.114450698741
R^2: 0.0668571818226
Mean Residual Deviance: 0.207466316266
Null degrees of freedom: 1451
Residual degrees of freedom: 1440
Null deviance: 322.824207989
Residual deviance: 301.241091219
AIC: 1862.91181644

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.24438568835
RMSE: 0.494353808876
MAE: 0.393732438242
RMSLE: 0.129567267121
R^2: 0.03611764032
Mean Residual Deviance: 0.24438568835
Null degrees of freedom: 175
Residual degrees of freedom: 164
Null deviance: 45.4899720344
Residual deviance: 43.0118811496
AIC: 277.481024028

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 0.212007029334
RMSE: 0.460442210635
MAE: 0.370032327697
RMSLE: 0.115690718364
R^2: 0.0464339446191
Mean Residual Deviance: 0.

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.3699967,0.0065473,0.3833716,0.3698760,0.3765940,0.3622156,0.3579263
mean_residual_deviance,0.2122576,0.0132232,0.2464497,0.2109187,0.2049418,0.2094392,0.1895384
mse,0.2122576,0.0132232,0.2464497,0.2109187,0.2049418,0.2094392,0.1895384
null_deviance,64.59902,3.7726512,73.65858,63.39339,65.73587,63.026875,57.18041
r2,0.0452233,0.0077007,0.0470470,0.0450960,0.0423469,0.0628529,0.0287737
residual_deviance,61.56684,3.406152,69.991714,60.53368,62.712196,59.061867,55.53475
rmse,0.4602812,0.0141198,0.496437,0.4592589,0.452705,0.4576453,0.4353601
rmsle,0.1155672,0.0052760,0.1288012,0.1134193,0.1121694,0.1170857,0.1063606


Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2019-06-20 18:17:22,0.000 sec,0,322.8242080,0.2223307


In [17]:
p=glm_gauss.model_performance(test)
p


ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 0.228126224293
RMSE: 0.477625611011
MAE: 0.375577898669
RMSLE: 0.128208435291
R^2: 0.0805553379968
Mean Residual Deviance: 0.228126224293
Null degrees of freedom: 166
Residual degrees of freedom: 155
Null deviance: 41.9546193291
Residual deviance: 38.097079457
AIC: 253.123486712


In [18]:
boston_glm = H2OGeneralizedLinearEstimator(lambda_search = True)
boston_glm.train(x, y , training_frame = train, validation_frame = valid)

# print the mse for the validation data
print(boston_glm.mse(valid=True))

glm Model Build progress: |███████████████████████████████████████████████| 100%
0.216392663354


In [19]:
print(boston_glm.mae(valid=True))

0.373016944991


In [20]:
# cross validation using H2OGeneralizedLinearEstimator

train, test = rating.split_frame(
    ratios = [ 0.897], destination_frames = ['rating1_train', 'rating1_test'], seed = 122)

In [21]:
print('%d/%d' % (train.nrows, test.nrows))

1609/186


In [22]:
boston_glm = H2OGeneralizedLinearEstimator(lambda_search = True)
boston_glm.train(x, y , training_frame = train, validation_frame = valid)

# print the mse for the validation data
print(boston_glm.mse(valid=True))
print(boston_glm.mae(valid=True))

glm Model Build progress: |███████████████████████████████████████████████| 100%
0.216736299549
0.37237589103


In [23]:
cars_gbm = H2OGeneralizedLinearEstimator(keep_cross_validation_models = True, nfolds = 5, seed = 122)

# then train your model
cars_gbm.train(x , y , training_frame = train)

# retrieve the cross-validation models
cars_gbm_cv_models = cars_gbm.cross_validation_models()

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [24]:
cars_gbm_cv_models

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1560987482141_19_cv_1


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.212140650337
RMSE: 0.46058728851
MAE: 0.369049977531
RMSLE: 0.115923631984
R^2: 0.0601942264189
Mean Residual Deviance: 0.212140650337
Null degrees of freedom: 1290
Residual degrees of freedom: 1284
Null deviance: 291.415085205
Residual deviance: 273.873579585
AIC: 1677.99633165

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.215896760922
RMSE: 0.464646920707
MAE: 0.372924832189
RMSLE: 0.119678170122
R^2: 0.0462919603027
Mean Residual Deviance: 0.215896760922
Null degrees of freedom: 317
Residual degrees of freedom: 311
Null deviance: 72.0875797093
Residual deviance: 68.6551699733
AIC: 430.965234847
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2019-06-20 18:17:23,0.000 sec,0,291.4309369,0.1811255


Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1560987482141_19_cv_2


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.207836135259
RMSE: 0.455890486037
MAE: 0.36507559078
RMSLE: 0.115564039489
R^2: 0.0556236709833
Mean Residual Deviance: 0.207836135259
Null degrees of freedom: 1296
Residual degrees of freedom: 1291
Null deviance: 285.440728604
Residual deviance: 269.563467431
AIC: 1657.13265377

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.238576232583
RMSE: 0.488442660486
MAE: 0.391186785095
RMSLE: 0.121893910739
R^2: 0.0446719665766
Mean Residual Deviance: 0.238576232583
Null degrees of freedom: 311
Residual degrees of freedom: 306
Null deviance: 78.0724211067
Residual deviance: 74.4357845658
AIC: 452.300932208
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2019-06-20 18:17:23,0.000 sec,0,285.4651057,0.1774177


Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1560987482141_19_cv_3


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.217063183884
RMSE: 0.46590040125
MAE: 0.376021110319
RMSLE: 0.117103741773
R^2: 0.0571489062631
Mean Residual Deviance: 0.217063183884
Null degrees of freedom: 1275
Residual degrees of freedom: 1270
Null deviance: 293.760726881
Residual deviance: 276.972622637
AIC: 1685.95590268

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.199195215086
RMSE: 0.446312911629
MAE: 0.352376867846
RMSLE: 0.115713117626
R^2: 0.0460051522661
Mean Residual Deviance: 0.199195215086
Null degrees of freedom: 332
Residual degrees of freedom: 327
Null deviance: 69.7721320131
Residual deviance: 66.3320066238
AIC: 421.727568176
Scoring History: 


...

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1560987482141_19_cv_4


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.208218270763
RMSE: 0.456309402448
MAE: 0.364531332189
RMSLE: 0.115734681681
R^2: 0.0532360428487
Mean Residual Deviance: 0.208218270763
Null degrees of freedom: 1288
Residual degrees of freedom: 1283
Null deviance: 283.484968968
Residual deviance: 268.393351013
AIC: 1649.36550861

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.238369549324
RMSE: 0.488231040926
MAE: 0.390564730852
RMSLE: 0.122420046206
R^2: 0.0401383477717
Mean Residual Deviance: 0.238369549324
Null degrees of freedom: 319
Residual degrees of freedom: 314
Null deviance: 80.1295073159
Residual deviance: 76.2782557836
AIC: 463.262075236
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2019-06-20 18:17:23,0.000 sec,0,283.5903703,0.1762526


Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1560987482141_19_cv_5


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.220154230724
RMSE: 0.469205957682
MAE: 0.375083632744
RMSLE: 0.119189967589
R^2: 0.0573287703329
Mean Residual Deviance: 0.220154230724
Null degrees of freedom: 1282
Residual degrees of freedom: 1275
Null deviance: 299.635619641
Residual deviance: 282.457878019
AIC: 1717.26952565

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.18594722696
RMSE: 0.431215986439
MAE: 0.351599937242
RMSLE: 0.106396825145
R^2: 0.0498855434557
Mean Residual Deviance: 0.18594722696
Null degrees of freedom: 325
Residual degrees of freedom: 318
Null deviance: 63.8589052976
Residual deviance: 60.6187959891
AIC: 394.720610525
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2019-06-20 18:17:23,0.000 sec,0,299.6448824,0.1862305


[, , , , ]

In [25]:
## performing GridSearch and using hyper params and opted for RandomDiscreate 
from h2o.grid.grid_search import H2OGridSearch

In [26]:
g= h2o.grid.H2OGridSearch(H2OGeneralizedLinearEstimator(family="gaussian", lambda_search = True),
                        hyper_params = {"alpha": [x1 * 0.01 for x1 in range(0,100)],},
                          search_criteria = { "strategy":"RandomDiscrete","max_models":7,"max_runtime_secs":30})

In [27]:
g.train(x, y, train, validation_frame = valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [28]:
g

                    alpha  \
0    [0.9500000000000001]   
1                  [0.84]   
2                   [0.8]   
3                  [0.79]   
4                  [0.73]   
5                   [0.5]   
6                  [0.37]   

                                                      model_ids  \
0  Grid_GLM_rating1_train_model_python_1560987482141_20_model_6   
1  Grid_GLM_rating1_train_model_python_1560987482141_20_model_5   
2  Grid_GLM_rating1_train_model_python_1560987482141_20_model_2   
3  Grid_GLM_rating1_train_model_python_1560987482141_20_model_1   
4  Grid_GLM_rating1_train_model_python_1560987482141_20_model_4   
5  Grid_GLM_rating1_train_model_python_1560987482141_20_model_3   
6  Grid_GLM_rating1_train_model_python_1560987482141_20_model_7   

    residual_deviance  
0   37.76406487656328  
1  37.833489452203224  
2  37.859923266951945  
3   37.86692345132296  
4  37.908019970313454  
5  38.145588720709405  
6  38.387096232753414  


In [29]:
pg=g.model_performance(test)
pg


ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 0.210427173005
RMSE: 0.458723416674
MAE: 0.367871340341
RMSLE: 0.120198541123
R^2: 0.151132763168
Mean Residual Deviance: 0.210427173005
Null degrees of freedom: 185
Residual degrees of freedom: 36
Null deviance: 46.5725553096
Residual deviance: 39.1394541789
AIC: 539.942622114

ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 0.210390362563
RMSE: 0.458683292221
MAE: 0.367858515305
RMSLE: 0.120188763948
R^2: 0.151281257194
Mean Residual Deviance: 0.210390362563
Null degrees of freedom: 185
Residual degrees of freedom: 37
Null deviance: 46.5725553096
Residual deviance: 39.1326074367
AIC: 537.910081919

ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 0.211904652493
RMSE: 0.460331024908
MAE: 0.36907111653
RMSLE: 0.120591428872
R^2: 0.145172582679
Mean Residual Deviance: 0.211904652493
Null degrees of freedom: 185
Residual degrees of freedom: 35
Null deviance: 46.5725553096
Residual devian

{u'Grid_GLM_rating1_train_model_python_1560987482141_20_model_1': ,
 u'Grid_GLM_rating1_train_model_python_1560987482141_20_model_2': ,
 u'Grid_GLM_rating1_train_model_python_1560987482141_20_model_3': ,
 u'Grid_GLM_rating1_train_model_python_1560987482141_20_model_4': ,
 u'Grid_GLM_rating1_train_model_python_1560987482141_20_model_5': ,
 u'Grid_GLM_rating1_train_model_python_1560987482141_20_model_6': ,
 u'Grid_GLM_rating1_train_model_python_1560987482141_20_model_7': }

In [ ]:
##rid_GLM_rating1_train_model_python_1560987482141_20_model_6 is saved 
##as the Residual deviance is less than the base model